<a href="https://colab.research.google.com/github/ihagoSantos/natural-language-processing/blob/main/text_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similaridade de Textos
Indica o quão "próximos" dois textos estão em significado ou na forma de escrita.

In [1]:
# Instalação das bibliotecas necessárias
!pip install nltk==3.8.1
!pip install unidecode==1.3.8
!pip install scikit-learn==1.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully uninstalled nltk-3.9.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2


In [2]:
# Importação das bibliotecas padrão
import datetime
import math
import re
import string
import sys
import warnings

# importação das bibliotecas de terceiros
import nltk

from nltk.corpus import stopwords

import sklearn
from sklearn.feature_extraction.text import CountVectorizer

from unidecode import unidecode

# Downloads do nltk
nltk.download('punkt')
nltk.download('stopwords')

# Configurações de comandos específicos
warnings.filterwarnings('ignore')

print("Pacotes importados com sucesso!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Pacotes importados com sucesso!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def preprocessa_texto(texto):
  """
  Preprocessa o texto fornecido realizando várias etapas de limpeza

  Etapas:
  1. Tokeniza o texto
  2. Converte os tokens para minusculo
  3. Remove stopwords em português
  4. Remove números dos tokens
  5. Exclui tokens que são pontuações
  6. Remove acentuações dos tokens

  Parâmetros:
  texto (str): O texto a ser preprocessado

  Retorna:
  list: Lista de tokens preprocessados
  """

  # Tokeniza o texto usando um padrão para capturar palavras e pontuações.
  padrao = r"\w+(?:'\w+)?|[^\w\s]"
  tokens_preprocessados = re.findall(padrao, texto)

  # Converte o texto para minúsculo para padronizar a capitalização
  tokens_preprocessados = [token.lower() for token in tokens_preprocessados]

  # Remove stopwords para reduzir o conjunto de tokens a palavras significativas
  portugues_stops = stopwords.words('portuguese')
  tokens_preprocessados = [token for token in tokens_preprocessados if token not in portugues_stops]

  # Remove números, pois geralmente não contribuem para o significado do texto
  tokens_preprocessados = [re.sub(r'\d+', '', token) for token in tokens_preprocessados if re.sub(r'\d+', '', token)]

  # Exclui tokens que são pontuações, pois raramente são úteis par análise de texto
  tokens_preprocessados = [token for token in tokens_preprocessados if token not in string.punctuation]

  # Remove acentuações para padronizar os tokens
  tokens_preprocessados = [unidecode(token) for token in tokens_preprocessados]

  return ' '.join(tokens_preprocessados)



In [4]:
texto_1 = preprocessa_texto("O gato comeu o rato")
texto_2 = preprocessa_texto("O rato comeu a comida do gato")
print(f"Textos preprocessados:\n{texto_1}\n{texto_2}")

Textos preprocessados:
gato comeu rato
rato comeu comida gato


# Similaridade de Jaccard

In [9]:
def similaridade_jaccard(a, b):
  """
  Calcula a similaridade de Jaccard entre duas listas.

  A similaridade de Jaccard é uma medida usada para comparar a semelhança e diversidade de conjuntos de amostras.
  O Coeficiente de similaridade de Jaccard é definido como o tamanho da interseção dividido pelo tamanho da união das amostras.

  Args:
  a: Lista de elementos
  b: Lista de elementos

  Returns:
  Um float representando a similaridade de Jaccard entre duas listas, que é o tamanho da interseção
  dos conjuntos dividido pelo tamanho da união dos conjuntos.

  Raises:
  ZeroDivisionError: Se a união das listas resultar em um conjunto vazio, o que levaria a uma divisão por zero.
  """

  interseccao = len(set.intersection(*[set(a), set(b)]))
  uniao = len(set.union(*[set(a), set(b)]))

  if uniao == 0: return 0

  return interseccao/uniao

In [10]:
corpus = [texto_1, texto_2]
tokens = [texto.split(" ") for texto in corpus]
print(tokens)

[['gato', 'comeu', 'rato'], ['rato', 'comeu', 'comida', 'gato']]


In [11]:
similaridade = similaridade_jaccard(tokens[0], tokens[1])
print(f"Similaridade de Jaccard entre os textos: {similaridade}")

Similaridade de Jaccard entre os textos: 0.75


# Similaridade de Distância Euclidiana

In [27]:
def similaridade_euclidiana(a, b):
  """
  Calcula a similaridade pela Distância Euclidiana entre dois vetores.

  A similaridade Euclidiana é determinada pela transformação da distância
  euclidiana entre dois pontos (vetores) em uma medida de similaridade.
  Esta transformação é realizada através da função exponencial, que mapeia a distância
  para um valor entre 0 e 1, onde valores próximos a 1 indicam alta similaridade
  e valores próximos a 0 indicam baixa similaridade.

  Args:
  a: Uma lista ou vetor de valores numéricos
  b: Outra lista ou vetor de valores numéricos, com a mesma dimensão de 'a'

  Returns:
  Um float representando a similaridade euclidiana entre os vetores 'a' e 'b'.

  Raises:
  ValueError: Se os vetores 'a' e 'b' têm dimensões diferentes.
  """

  if len(a) != len(b):
    return ValueError("Os vetores 'a' e 'b' devem ter a mesma dimensão")

  distancia = math.sqrt(sum(math.pow(x - y, 2) for x, y in zip(a, b) ) )
  return math.exp(-distancia)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
vetorizador = CountVectorizer(stop_words=None)
frequencias = vetorizador.fit_transform(corpus)

print(f"Tokens: {vetorizador.get_feature_names_out()}")
print(f"Frequências: {frequencias.toarray()}")

Tokens: ['comeu' 'comida' 'gato' 'rato']
Frequências: [[1 0 1 1]
 [1 1 1 1]]


In [31]:
similaridade = similaridade_euclidiana(frequencias.toarray()[0], frequencias.toarray()[1])
print(f"Similaridade Euclidiana: {similaridade}")

Similaridade Euclidiana: 0.36787944117144233


# Similaridade de Cosseno

In [32]:
def norma_vetor(x):
  """
  Calcula a norma (ou magnitude) de um vetor.

  A norma é calculada como a raiz quadrada da soma dos quadrados de cada
  elemento do vetor. Essa função é comumente usada em operações de álgebra linear
  e análise de vetores.

  Args:
  x: Uma lista de valores numéricos representando um vetor.

  Returns:
  Um float representando a norma do vetor, arredondado para 3 casas decimais
  """

  return round(math.sqrt(sum(a * a for a in x)), 3)

def similaridade_cosseno(a, b):
  """
  Calcula a similaridade de cosseno entre dois vetores.

  A similaridade de cosseno é uma medida que calcula o ângulo entre dois vetores
  no espaço multidimensional, sendo usada frequentemente para medir a semelhança
  entre dois vetores.
  O resultado varia entre -1 e 1, onde 1 indica vetores idênticos, 0 indica ortogonalidade,
  e -1 indica vetores dialmente opostos.

  Args:
  a: Uma lista de valores numéricos representando o primeiro vetor.
  b: Uma lista de valores numéricos representando o segundo vetor.

  Return:
  float: Representa a similaridade de cosseno entre os vetores 'a' e 'b'.
  """
  if len(a) != len(b):
    return ValueError("Os vetores 'a' e 'b' devem ter a mesma dimensão")

  numerador = sum(x * y for x,y in zip(a,b))
  denominador = norma_vetor(a) * norma_vetor(b)

  return numerador / float(denominador)

In [33]:
similaridade = similaridade_cosseno(frequencias.toarray()[0], frequencias.toarray()[1])
print(f"Similaridade de Cosseno: {similaridade}")

Similaridade de Cosseno: 0.8660508083140878
